In [1]:
import driptorch as dt
import json
from IPython.display import GeoJSON

In [2]:
# Burn Unit Parameters
burn_unit_geojson : 'CWLStringInput' = '{"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-114.44869995117188,47.088504171925706],[-114.44470882415771,47.08745225315146],[-114.44342136383057,47.09066638416644],[-114.44496631622313,47.09236102969754],[-114.44633960723877,47.0924194647886],[-114.45281982421875,47.089205439567344],[-114.45153236389159,47.08815353464254],[-114.44869995117188,47.088504171925706]]]}}]}'
burn_unit_firing_direction : 'CWLStringInput' = '217'
burn_unit_control_line_buffer : 'CWLStringInput' = '10'
burn_unit_downwind_buffer : 'CWLStringInput' = '20'

# Igniter Parameters
# Dash length = 0 & Gap length > 0 : Point igniter. 
# Gap length = 0 & Dash length = 0: Continuous igniter
number_of_igniters : 'CWLIntInput' = 4
igniters_same_velocity : 'CWLStringInput' = 'True'
igniter_velocities : 'CWLStringInput' = '[1, 1, 1, 1]'
igniter_line_dash_lengths : 'CWLStringInput' = '[10, 10, 10, 10]' 
igniter_line_gap_lengths : 'CWLStringInput' = '[50, 10, 50, 10]' 

# Firing Parameters
firing_technique : 'CWLStringInput' = 'strip' # strip, flank, ring, head, back, inferno
firing_spacing : 'CWLStringInput' = 100 # Staggering distance in meters between igniters within a heat
firing_depth : 'CWLStringInput' = 100 # Horizontal distance in meters between igniters and heats
firing_heat_depth : 'CWLStringInput' = None # Depth in meters between igniter heats. If None, heat_depth is equal to igniter depth
firing_heat_delay : 'CWLStringInput' = None # Delay in seconds between igniter heats.
firing_side : 'CWLStringInput' = 'right' # Side of the wind vector to start the ignition. 'right' or 'left'
firing_offset : 'CWLStringInput' = None # Offset distance in meters from the unit boundary
firing_clockwise : 'CWLStringInput' = None # Toggle path travel direction relative to the unit bounds (for Ring & Backing firing)

# Output Parameters
output_time_offset : 'CWLIntInput' = 25 # Time offset to add to the ignition times. Defaults to 0
output_resolution : 'CWLIntInput' = 1 # Horizontal resolution of QUIC-fire domain (in meters)
output_epsg : 'CWLStringInput' = None # EPSG code for the destination projection
output_file : 'CWLFilePathOutput' = "./qf_ignition.dat"

In [3]:
# BURN UNIT

if not burn_unit_geojson:
    raise ValueError('No geojson provided for burn unit')
burn_unit_geojson = json.loads(burn_unit_geojson)

# Create a burn unit from a GeoJSON feature collection with a wind direction
if burn_unit_firing_direction:
    burn_unit = dt.BurnUnit.from_json(burn_unit_geojson, firing_direction=float(burn_unit_firing_direction))
else:
    burn_unit = dt.BurnUnit.from_json(burn_unit_geojson)

# GeoJSON(burn_unit.to_json())

# Add Buffer Areas
if burn_unit_control_line_buffer:
    burn_unit_firing_area = burn_unit.buffer_control_line(float(burn_unit_control_line_buffer))
if burn_unit_downwind_buffer:
    burn_unit_firing_area = burn_unit_firing_area.buffer_downwind(float(burn_unit_downwind_buffer))

fuel_removal_area = burn_unit.difference(burn_unit_firing_area)

# GeoJSON(burn_unit_firing_area.to_json())
# GeoJSON(fuel_removal_area.to_json())

# TODO: 
# Do we need to store the burn unit firing area & fuel removal area as outputs ?

In [4]:
# PERSONNEL (IGNITERS)
igniter_velocities = json.loads(igniter_velocities)
assert(len(igniter_velocities) == number_of_igniters)

igniter_line_dash_lengths = json.loads(igniter_line_dash_lengths)
assert(len(igniter_line_dash_lengths) == number_of_igniters)

igniter_line_gap_lengths = json.loads(igniter_line_gap_lengths)
assert(len(igniter_line_gap_lengths) == number_of_igniters)

igniters_same_velocity = (igniters_same_velocity.lower() == "true")

ignition_crew = dt.IgnitionCrew(same_velocity=igniters_same_velocity)
for i in range(0, number_of_igniters):
    igniter_velocity = igniter_velocities[i]
    igniter_line_dash_length = igniter_line_dash_lengths[i]
    igniter_line_gap_length = igniter_line_gap_lengths[i]
    igniter =  dt.Igniter(igniter_velocity, dash_length=igniter_line_dash_length, gap_length=igniter_line_gap_length)
    ignition_crew.add_igniter(igniter)

# TODO:
# Do we need to store the ignition crew details as outputs ?

In [5]:
# FIRING TECHNIQUES
if firing_technique == "strip":
    firing = dt.firing.Strip(burn_unit_firing_area, ignition_crew)
    firing_pattern = firing.generate_pattern(
        spacing=float(firing_spacing) if firing_spacing else None, 
        depth=float(firing_depth) if firing_depth else None,
        heat_depth=float(firing_heat_depth) if firing_heat_depth else None,
        heat_delay=float(firing_heat_delay) if firing_heat_delay else None,
        side=firing_side if firing_side else None
    ) 
elif firing_technique == "flank":
    firing = dt.firing.Flank(burn_unit_firing_area, ignition_crew)
    firing_pattern = firing.generate_pattern(
        spacing=float(firing_spacing) if firing_spacing else None, 
        depth=float(firing_depth) if firing_depth else None,
        heat_depth=float(firing_heat_depth) if firing_heat_depth else None,
        heat_delay=float(firing_heat_delay) if firing_heat_delay else None,
        side=firing_side if firing_side else None
    ) 
elif firing_technique == "ring":
    firing = dt.firing.Ring(burn_unit_firing_area, ignition_crew)
    firing_pattern = firing.generate_pattern(
        offset=float(firing_offset) if firing_offset else None
    ) 
elif firing_technique == "head":
    firing = dt.firing.Head(burn_unit_firing_area, ignition_crew)
    firing_pattern = firing.generate_pattern(
        offset=float(firing_offset) if firing_offset else None,
        clockwise=(firing_clockwise.lower() == "true") if firing_clockwise else None
    )
elif firing_technique == "back":
    firing = dt.firing.Back(burn_unit_firing_area, ignition_crew)
    firing_pattern = firing.generate_pattern(
        offset=float(firing_offset) if firing_offset else None,
        clockwise=(firing_clockwise.lower() == "true") if firing_clockwise else None
    )
elif firing_technique == "inferno":
    firing = dt.firing.Inferno(burn_unit_firing_area, ignition_crew)
    firing_pattern = firing.generate_pattern()


In [6]:
# OUTPUTS

if not firing_pattern:
    raise ValueError('No firing pattern generated')

# Write the pattern to a QUIC-Fire ignition file
qf_ignition_file = firing_pattern.to_quicfire(
    burn_unit, 
    filename=output_file, 
    time_offset=output_time_offset,
    resolution=output_resolution,    
    dst_epsg=output_epsg
)

In [ ]:
# SHOW MAP
'''
map = dt.Map(burn_unit)

# Optionally add the firing and blackline areas
map.add_firing_area(burn_unit_firing_area)
map.add_blackline_area(fuel_removal_area)

# Add the timed ignition pattern
map.add_pattern(firing_pattern)

# Show the map interactively in a notebook
map.show()
'''